In [ ]:
%load_ext autoreload
%autoreload 2

import functools
import os
import cv2
import matplotlib.pylab as plt
import numpy as np

import PIL
import sys
import glob
import subprocess
import pandas as pd
import random
sys.path.append('../py')

import ai_utils
from image_utils import load_image, show_image, save_image
import image_utils as iu
import time
import multiprocess as mp


from layout_utils import translate_data, put_text_wrap, get_coords
from video_utils import run_ebsynth, join_videos, reverse_video

In [ ]:
base_path = "/Users/iman/github/avignon/"    
project_dir =  "projects/graphite/"
input_dir = os.path.join(base_path, project_dir, "graphite_rev")
old_input_dir = os.path.join(base_path, project_dir, "style_rev")
mask_dir = os.path.join(base_path, project_dir, "graphite_rev")
output_dir = os.path.join(base_path, project_dir, "output")
styles_dir = os.path.join(base_path, project_dir, "styles")
temp_dir = os.path.join(base_path, project_dir, "temp")
styles_dir = os.path.join(base_path, project_dir, "styles")

In [ ]:
input_files = glob.glob(os.path.join(input_dir, '*'))
input_files.sort()

old_input_files = glob.glob(os.path.join(old_input_dir, '*'))
old_input_files.sort()

content_img_size = (1920, 1080)
style_img_size = (1920, 1080)

In [ ]:
collected_csv_path = os.path.join(base_path, "questions/q_and_a_truth.csv")
# collected_csv_path = os.path.join(base_path, "questions/q_and_a_audience.csv")
collected_df = translate_data(collected_csv_path)

In [ ]:
static_csv_path = os.path.join(base_path, "questions/static_q_and_a.csv")
static_df = translate_data(static_csv_path)

In [ ]:
joined_df = pd.concat([collected_df, static_df]) 


In [ ]:
joined_df['question number'].max()+1

In [ ]:
joined_df["ratio"].sum()

In [ ]:
def get_style(input_file, out_dir, segments, joined_df, grid_rows=6, grid_cols=2, fsize_max=20, fsize_min=3, fstroke_max=40, colour_max=140, ratio_max=7.91, show=False):
    input_image = load_image(input_file)
    
    height = input_image.shape[0]
    width = input_image.shape[1]

    blank_image = np.ones((height, width, 3), np.uint8) 
    blank_image.fill(0)

    cell_width = width // 2
    cell_height = height // 6

    r_sum = joined_df['ratio'].sum()
    r_truth = 7.91
    r_ratio = r_sum/r_truth
    r_ratio = max(r_ratio,0.85)
    r_ratio = min(r_ratio,1.75)

    for segment in segments:
        seg_joined_df = joined_df[joined_df['question number'] == segment]
        i_row, i_col = get_coords(segment, rows=grid_rows, cols=grid_cols)
    
        x = i_col * cell_width - 50
        y = i_row * cell_height 

        adj_x = x
        adj_y = y + cell_height
        
        for index, row in seg_joined_df.iterrows():
            text = row['question'].replace("sex", '')
            ratio = row['ratio']
            put_text_wrap(img=blank_image, 
                        text=text, 
                        org=(adj_x, adj_y-300),  
                        font=cv2.FONT_HERSHEY_SIMPLEX, 
                        font_scale=max(15 * ratio, 2), 
                        color=(10, 5, 10), 
                        thickness=int(max((ratio*8),1)), 
                        line_spacing=int((1/ratio)*100), 
                        max_width=500)

        for index, row in seg_joined_df.iterrows():
            text = row['numbers']
            ratio = row['ratio']
            green = colour_max * ratio /3
            # print(blue)
            blue = 255 *0*(1-ratio)
            red = 255 * 0.5*(1-ratio)
            cv2.putText(img=blank_image,
                text=text,
                org=(adj_x,adj_y),
                fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                fontScale=max(fsize_max*ratio,fsize_min),
                color=(red,green,blue),
                thickness=int(max(fstroke_max*(ratio),fsize_min)))
            

    temp_file_path = os.path.join(out_dir, os.path.basename(input_file))
    s_image = iu.overlay(blank_image,input_image)
    # s_image = iu.edges(s_image,min_g=1,max_g=4+(r_truth - r_sum)*4)
    s_image = iu.edges(s_image,min_g=1,max_g=4)
    s_image = cv2.cvtColor(s_image, cv2.COLOR_GRAY2BGR)
    s_image = iu.overlay(input_image, s_image, 0.7*r_ratio)
    s_image = iu.overlay(blank_image, s_image, 0.3*r_ratio)
    s_image = iu.adjust_brightness_contrast(s_image, brightness=50, contrast=50)
    # s_image= iu.apply_filter(s_image, 1, 1, r_ratio)
    if show:
        show_image(s_image, fig_size=(10,5))
    save_image(temp_file_path, s_image)
    return temp_file_path

In [ ]:
# question_segments = 12
# frames_per_segment = 50
# segments = []
# for i in range(0,question_segments):
#     segments.append(i)
#     style_file = get_style(input_file=input_files[i*frames_per_segment], out_dir=styles_dir, segments=segments, joined_df=joined_df, show=True)

In [ ]:
# Gen sample
sample_n = 492
segment = sample_n//50
segments=range(0, sample_n//(segment+1))
style_file = get_style(input_file=input_files[segment*50], out_dir=temp_dir, segments=segments, joined_df=joined_df, show=True)
temp_file = get_style(input_file=input_files[sample_n], out_dir=temp_dir, segments=segments, joined_df=joined_df)
# show_image(load_image(temp_file))
mask_file = input_files[sample_n]
out_dir = "/Users/iman"
temp_file = os.path.join(temp_dir, os.path.basename(input_files[sample_n]))

guide_1_weight = 1000000
guide_2 = None
guide_2_weight = 10000

start_time = time.time()
output_file = run_ebsynth(uniformity=1,style=style_file, guide_2=guide_2, guide_1_weight=guide_1_weight, guide_2_weight=guide_2_weight, input_file=temp_file, guide_1=mask_file, out_dir=out_dir)
elapsed_time = time.time() - start_time
print("Elapsed time: {:.2f} seconds".format(elapsed_time))
print(output_file)
out_image = load_image(output_file)
out_image = iu.adjust_brightness_contrast(out_image, brightness=10, contrast=10)
hsv_image = cv2.cvtColor(out_image, cv2.COLOR_BGR2HSV)

# Increase the saturation
saturation_factor = 1.8  # Adjust this value as per your requirement
hsv_image[..., 1] = hsv_image[..., 1] * saturation_factor

# Convert HSV back to BGR
result_image = cv2.cvtColor(hsv_image, cv2.COLOR_HSV2BGR)
# save_image(output_file, result_image)
show_image(result_image,(12,5))
# break
# show_image(load_image(output_frames[j]), fig_size=(2.5,5))


In [ ]:
output_frames = []
segments = []
base_style_path = input_files[0]
continue_from = 0
frames_per_segment = 50
question_segments = 12
for i in range(0,question_segments):
    gen_style = False
    segments.append(i)
    for j in range(i*frames_per_segment,(i*frames_per_segment)+frames_per_segment, 1):
        if j < continue_from:
            continue
        file_idx = j
        input_file = input_files[file_idx]
        temp_file = os.path.join(temp_dir, os.path.basename(input_files[file_idx]))
        mask_file = input_files[file_idx]
        temp_file = get_style(input_file=input_file, out_dir=temp_dir, segments=segments, joined_df=joined_df)
        
        if continue_from:
            style_file = get_style(input_file=input_files[j], out_dir=temp_dir, segments=segments, joined_df=joined_df)
            gen_style = True
            continue_from = 0
        if not gen_style:
            style_file = temp_file
            gen_style = True


        guide_1_weight = 1000000
        # guide_2 = old_input_files[j]
        guide_2 = None

        guide_2_weight = 10000

        start_time = time.time()
        output_file = run_ebsynth(uniformity=1,style=style_file, guide_2=guide_2, guide_1_weight=guide_1_weight, guide_2_weight=guide_2_weight, input_file=temp_file, guide_1=mask_file, out_dir=output_dir)
        elapsed_time = time.time() - start_time
        print("Elapsed time: {:.2f} seconds".format(elapsed_time))

        out_image = load_image(output_file)
        out_image = iu.adjust_brightness_contrast(out_image, brightness=10, contrast=10)
        hsv_image = cv2.cvtColor(out_image, cv2.COLOR_BGR2HSV)

# Increase the saturation
        saturation_factor = 1.8  # Adjust this value as per your requirement
        hsv_image[..., 1] = hsv_image[..., 1] * saturation_factor

        # Convert HSV back to BGR
        result_image = cv2.cvtColor(hsv_image, cv2.COLOR_HSV2BGR)
        save_image(output_file, result_image)

        # break
        # show_image(load_image(output_frames[j]), fig_size=(2.5,5))


In [ ]:
frames_per_segment = 50
frames_directory = output_dir
output_video_f_path = '/Users/iman/output_video_f.mp4'
output_video_r_path = '/Users/iman/output_video_r.mp4'

output_files = glob.glob(os.path.join(frames_directory, '*.png'))
output_files.sort()

segments = 12
frames_per_image = 3
interpolation_factor = 0.5


do_blur = 0
# kernel_sizes = [num for num in range(2, 1000) if num % 2 != 0]
kernel_sizes = [1,3,5]
# skip_percentage = 5
color_percentage = 80

max_contrast= 0
max_brightness= 100
skip_percentage = 0

cvt = None
question_segments = 12
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_video_all = cv2.VideoWriter(output_video_f_path, fourcc, 30, (1080, 1920))  

for i in range(0, question_segments):
    output_video_path = f'/Users/iman/output_video_{i}.mp4'
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    output_video_short = cv2.VideoWriter(output_video_path, fourcc, 30, (1080, 1920))  
    for j in range(i*frames_per_segment, (i+1)*frames_per_segment):
        current_frame = load_image(output_files[j], cvt=cvt) 
        next_frame =  load_image(output_files[min(len(output_files)-1,j+1)], cvt=cvt) 

        interpolated_frame = cv2.addWeighted(current_frame, interpolation_factor, next_frame, 1 - interpolation_factor, 0)

        filter_n = (1/12)*(i+1)
        filter_n = max(filter_n, 0.25)
        interpolated_frame = (filter_n*interpolated_frame.astype(np.float32)).astype(np.uint8)
        
        random_number = random.randint(0,100)
        if random_number < skip_percentage:
            do_blur = do_blur + 3

        if do_blur:
            kernel_size = (kernel_sizes[min(len(kernel_sizes)-1,do_blur)], kernel_sizes[min(len(kernel_sizes)-1,do_blur)])  # Adjust the kernel size based on the desired blur effect
            interpolated_frame = cv2.GaussianBlur(interpolated_frame, kernel_size, 0)
            do_blur = do_blur - 1

        for _ in range(frames_per_image):
            output_video_short.write(interpolated_frame)
            output_video_all.write(interpolated_frame)
    
    output_video_short.release()
output_video_all.release()



In [ ]:

output_video_forward_path = '/Users/iman/output_video_11.mp4'
output_video_reverse_path = '/Users/iman/output_video_11_r.mp4'
output_video_joined_path = '/Users/iman/output_video_11_joined.mp4'

In [ ]:
reverse_video(output_video_forward_path, output_video_reverse_path)


In [ ]:
join_videos(output_video_forward_path, output_video_reverse_path, output_video_joined_path)
